In [1]:
import pandas as pd
import requests
import os
from nba_api.stats.static import players,teams


start_year=1997
end_year=2025

frames=[]
for year in range(start_year,end_year):
    frame=pd.read_csv(str(year)+'.csv')
    if year>=2014:
        frame2=pd.read_csv(str(year)+'_2.csv')
        joined_columns = set(frame2.columns) - set(frame.columns)
        joined_columns = list(joined_columns)
        joined_columns.append('PLAYER_ID')
        joined_columns.append('date')
        frame2 = frame2[joined_columns]
        frame=frame.merge(frame2,on=['PLAYER_ID','date'],how='left')
        frame3=pd.read_csv(str(year)+'_3.csv')
        
        joined_columns = set(frame3.columns) - set(frame.columns)
        joined_columns = list(joined_columns)

        joined_columns.append('PLAYER_ID')
        joined_columns.append('date')
        frame3 = frame3[joined_columns]
        frame=frame.merge(frame3,on=['PLAYER_ID','date'],how='left')
    frames.append(frame)
df=pd.concat(frames)
print(df.columns)
df.drop_duplicates(inplace=True)
print(len(df))
def get_dates(start_year,end_year):
    dates=[]
    for year in range(start_year,end_year):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = '../team/'+str(year)+'ps/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['PLAYER_ID','TEAM_ID','HTM','VTM','GAME_DATE']]
                df.rename(columns={'GAME_DATE':'date'},inplace=True)
                df.drop_duplicates(inplace=True)
                dates.append(df)
    return pd.concat(dates)
    
dates=get_dates(start_year,end_year)
df=df.merge(dates,how='left').reset_index(drop=True)
home=df[df.HTM==df.TEAM_ABBREVIATION].reset_index(drop=True)
home.drop(columns='HTM',inplace=True)

home.rename(columns={'VTM':'opp_team'},inplace=True)


home

away=df[df.VTM==df.TEAM_ABBREVIATION].reset_index(drop=True)
away.drop(columns='VTM',inplace=True)
away.rename(columns={'HTM':'opp_team'},inplace=True)

away

none=df[df.HTM.isna()].reset_index(drop=True)
#none.drop(columns=['VTM','HTM'],inplace=True)
print(none)
df=pd.concat([home,away,none])


oppframe=df[['TEAM_ID','date','opp_team']]
oppframe.dropna(subset='opp_team',inplace=True)
oppframe.drop_duplicates(inplace=True)
df.drop(columns='opp_team',inplace=True)

#df=df.merge(oppframe)
df

df=df.merge(oppframe,on=['TEAM_ID','date'],how='left')
df

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT',
       ...
       'LT_06_PCT', 'FGM_LT_06', 'PLAYER_POSITION', 'FREQ', 'CATCH_SHOOT_FGA',
       'CATCH_SHOOT_FG3M', 'CATCH_SHOOT_EFG_PCT', 'CATCH_SHOOT_FG3A',
       'CATCH_SHOOT_FGM', 'CATCH_SHOOT_FG3_PCT'],
      dtype='object', length=324)
46630
     PLAYER_ID             PLAYER_NAME  NICKNAME     TEAM_ID  \
0           76         Cedric Ceballos    Cedric  1610612756   
1          916            Corie Blount     Corie  1610612747   
2          112      Dontonio Wingfield  Dontonio  1610612757   
3          695       Eldridge Recasner  Eldridge  1610612737   
4         1114           Jaren Jackson     Jaren  1610612764   
...        ...                     ...       ...         ...   
3916   1641716           Jarace Walker    Jarace  1610612754   
3917    203648  Thanasis Antetokounmpo  Thanasis  1610612749   
3918   1630188             Jalen Smith     Jalen  161061

/tmp/ipykernel_7770/1380577989.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppframe.dropna(subset='opp_team',inplace=True)
/tmp/ipykernel_7770/1380577989.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppframe.drop_duplicates(inplace=True)


PLAYER_ID             PLAYER_NAME   NICKNAME     TEAM_ID  \
0           673          Alan Henderson       Alan  1610612737   
1           183              Bison Dele      Bison  1610612741   
2             2             Byron Scott      Byron  1610612747   
3           363      Christian Laettner  Christian  1610612737   
4           241          Darrin Hancock     Darrin  1610612737   
...         ...                     ...        ...         ...   
46625   1641716           Jarace Walker     Jarace  1610612754   
46626    203648  Thanasis Antetokounmpo   Thanasis  1610612749   
46627   1630188             Jalen Smith      Jalen  1610612754   
46628   1641716           Jarace Walker     Jarace  1610612754   
46629   1630579            Jericho Sims    Jericho  1610612752   

      TEAM_ABBREVIATION   AGE  GP  W  L  W_PCT  ...   FREQ  CATCH_SHOOT_FGA  \
0                   ATL  24.0   1  1  0    1.0  ...    NaN              NaN   
1                   CHI  28.0   1  1  0    1.0  ...    NaN              NaN   
2                   LAL  36.0   1  1  0    1.0  ...    NaN              NaN   
3                   ATL  27.0   1  1  0    1.0  ...    NaN              NaN   
4                   ATL  25.0   1  1  0    1.0  ...    NaN              NaN   
...                 ...   ...  .. .. ..    ...  ...    ...              ...   
46625               IND  20.0   1  1  0    1.0  ...  0.167              0.0   
46626               MIL  31.0   1  0  1    0.0  ...  1.000              0.0   
46627               IND  24.0   1  1  0    1.0  ...  0.294              0.0   
46628               IND  20.0   1  1  0    1.0  ...  0.083              0.0   
46629               NYK  25.0   1  0  1    0.0  ...  0.400              0.0   

       CATCH_SHOOT_FG3M  CATCH_SHOOT_EFG_PCT  CATCH_SHOOT_FG3A  \
0                   NaN                  NaN               NaN   
1                   NaN                  NaN               NaN   
2                   NaN                  NaN               NaN   
3                   NaN                  NaN               NaN   
4                   NaN                  NaN               NaN   
...                 ...                  ...               ...   
46625               NaN                  NaN               NaN   
46626               NaN                  NaN               NaN   
46627               NaN                  NaN               NaN   
46628               NaN                  NaN               NaN   
46629               NaN                  NaN               NaN   

       CATCH_SHOOT_FGM  CATCH_SHOOT_FG3_PCT  HTM  VTM  opp_team  
0                  NaN                  NaN  NaN  NaN       DET  
1                  NaN                  NaN  NaN  NaN       WAS  
2                  NaN                  NaN  NaN  NaN       POR  
3                  NaN                  NaN  NaN  NaN       DET  
4                  NaN                  NaN  NaN  NaN       DET  
...                ...                  ...  ...  ...       ...  
46625              0.0                  NaN  NaN  NaN       MIL  
46626              0.0                  NaN  NaN  NaN       IND  
46627              0.0                  NaN  NaN  NaN       NYK  
46628              0.0                  NaN  NaN  NaN       NYK  
46629              0.0                  NaN  NaN  NaN       IND  

[46630 rows x 327 columns]

In [2]:

df['team']=df['TEAM_ABBREVIATION']
teammap=dict(zip(df['TEAM_ABBREVIATION'],df['TEAM_ID']))


player_index = df[['PLAYER_NAME','PLAYER_ID','team','TEAM_ID','opp_team','year']]

player_index['opp_id']=player_index['opp_team'].map(teammap)
player_index.drop_duplicates(inplace=True)
player_index.to_csv('series_index_players.csv',index=False)

/tmp/ipykernel_7770/1892710818.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_index['opp_id']=player_index['opp_team'].map(teammap)
/tmp/ipykernel_7770/1892710818.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_index.drop_duplicates(inplace=True)


In [3]:
none2=df[df.opp_team.isna()]
df=df.dropna(subset='opp_team')
teammap=dict(zip(df['TEAM_ABBREVIATION'],df['TEAM_ID']))

df['opp_id']=df['opp_team'].map(teammap)
df.to_csv('playoff_data.csv',index=False)

/tmp/ipykernel_7770/2211292126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['opp_id']=df['opp_team'].map(teammap)


In [4]:
df[df.TEAM_ABBREVIATION=='DET']
df['team']=df['TEAM_ABBREVIATION']

/tmp/ipykernel_7770/2916735606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team']=df['TEAM_ABBREVIATION']


In [5]:
df.sort_values(by='date',inplace=True)

df['series_key']=df['team']+'_'+df['opp_team']+'_'+df['year'].astype(str)


/tmp/ipykernel_7770/3823595001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='date',inplace=True)
/tmp/ipykernel_7770/3823595001.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['series_key']=df['team']+'_'+df['opp_team']+'_'+df['year'].astype(str)


In [6]:

series_index=df[['series_key','team','opp_team','TEAM_ID','opp_id','year']]
series_index.drop_duplicates(inplace=True)
series_index.to_csv('series_index.csv',index=False)

/tmp/ipykernel_7770/711117300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series_index.drop_duplicates(inplace=True)


In [7]:
for key in df['series_key'].unique():
    series=df[df.series_key==key]
    series.to_csv('data/'+key+'.csv',index=False)